In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import torchvision
from torch.nn.parameter import Parameter
from torch import Tensor

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tarfile
import os
import csv
import shutil

In [ ]:
# define new activation function
class LP_ReLU1(nn.Module):
    
    def __init__(self, in_features, alpha = None):
        '''
        Initialization.
        INPUT:
            - in_features: shape of the input
            - aplha: trainable parameter
            aplha is initialized with zero value by default
        '''
        super(LP_ReLU1,self).__init__()
        self.in_features = in_features

        # initialize alpha
        if alpha == None:
            self.alpha = Parameter(torch.tensor(6.0)) # create a tensor out of alpha
        else:
            self.alpha = Parameter(torch.tensor(alpha)) # create a tensor out of alpha
            
        self.alpha.requiresGrad = True # set requiresGrad to true!

    def forward(self, x:Tensor)->Tensor:
        '''
        Forward pass of the function.
        Applies the function to the input elementwise.
        '''
        if (x <= 0):
            return 0

        if (x <= self.alpha and x > 0):
            return x

        if (x > self.alpha):
            return (self.alpha + 0.05*(x - self.alpha))



In [28]:
class log_act(nn.Module):
    def __init__(self, alpha):
        super(log_act, self).__init__()
        # self.positive_flag = positive_flag
        self.a = alpha
        # self.C1 = beta
    def forward(self, x):
        # x = x.detach().numpy()
        x=x.clone().detach().requires_grad_(True)

        if (x.all() <= 0.0):
            out = 0

        if (x.all() <= self.a and x.all() > 0):
            out = x

        if (x.all() > self.a):
            out = (self.a + 0.05*(x - self.a))
        out = torch.tensor(out)
        return out


In [29]:
class log_act_helper(nn.Module):
    def __init__(self, alpha):
        super(log_act_helper, self).__init__()
        self.a = alpha
        
    def forward(self, x):
        x0 = x.clone().detach()
        f1 = log_act(alpha=self.a)
        out = f1(x0)
        # f2 = log_act(alpha=self.a)
        # out2 = f2(x0)
        # out = torch.where(np.greater(x0, 0), out1, out2)
        # out = torch.tensor(out)
        return out


In [30]:

x = torch.rand(3, 3, 448, 448)
conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                           bias=False)
x = conv1(x)
bn1 = nn.BatchNorm2d(64)
x = bn1(x)
activation = log_act_helper(alpha=0.2)
out = activation(x)
print(out.shape)


torch.Size([3, 64, 224, 224])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
